In [1]:
import pandas as pd  
from sklearn.model_selection import train_test_split
import xgboost as xgb
import os
import math

from xai4chem import MorganFingerprint, RDKitDescriptor, DatamolDescriptor, AccFgFingerprint
from xai4chem import Explainer
from xai4chem.supervised import Regressor, Classifier

#data = pd.read_csv("../data/plasmodium_falciparum_3d7_ic50.csv") #data path
data = pd.read_csv("/home/jason/Downloads/h3d_solubility_74_July2025_preds.csv")

2025-12-18 22:53:51.941457: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-18 22:53:52.673358: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Imported lazytransform v1.16. 

Imported featurewiz 0.5.7. Use the following syntax:
    >>> wiz = FeatureWiz(feature_engg = '', nrows=None, transform_target=True,
        		category_encoders="auto", auto_encoders='VAE', ae_options={},
        		add_missing=False, imbalanced=False, verbose=0)
    >>> X_train_selected, y_train = wiz.fit_transform(X_train, y_train)
    >>> X_test_selected = wiz.transform(X_test)
    >>> selected_features = wiz.features
    


In [2]:
# Extract SMILES and target values
smiles = data["smiles"]
target = data["activity"] #target value's column_name
#log_target = [math.log10(val) for val in data["exp"]]
#data.loc[:, "log_exp"] = log_target
#target = data["log_exp"]

# Split data into training and test sets
smiles_train, smiles_valid, y_train, y_valid = train_test_split(smiles, target, test_size=0.2, random_state=42)

# Reset indices
smiles_train.reset_index(drop=True, inplace=True)
smiles_valid.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_valid.reset_index(drop=True, inplace=True)

In [3]:
#descriptor = MorganFingerprint() #DatamolDescriptor() #RDKitDescriptor() #MordredDescriptor()
descriptor = DatamolDescriptor()
#descriptor = AccFgFingerprint()

# Fit the descriptor to training data
descriptor.fit(smiles_train)

# Transform the data
smiles_train_transformed = descriptor.transform(smiles_train)
smiles_valid_transformed = descriptor.transform(smiles_valid)

100%|██████████████████████████████████████| 1145/1145 [00:02<00:00, 383.59it/s]


In [4]:
# Specify the output folder where evaluation metrics and interpretability plots will be saved.
output_folder = "/home/jason/JHlozek_code/xai4chem_jhlozek/notebooks/h3d_solubility_July2025_datamol_AI2050"
explainer_path = os.path.join(output_folder, "regressor.joblib")

# Specify fingerprints to use: 'morgan' / 'accfg' / 'datamol' for Morgan Bits / Functional Groups / Global Properties respectively
regressor = Regressor(output_folder, k=2048, fingerprints='datamol') 

In [5]:
# Train the model
regressor.fit(smiles_train_transformed, y_train)

# Evaluate the model
regressor.evaluate(smiles_valid_transformed, smiles_valid, y_valid)

[I 2025-12-18 22:54:20,228] A new study created in memory with name: no-name-582bfda7-59f4-4a20-a5ce-ffa978befb80


Number of input features is less than or equal to 2048. Using all features.


[I 2025-12-18 22:54:21,801] Trial 0 finished with value: 0.18085571851678933 and parameters: {'lambda': 1, 'alpha': 0, 'gamma': 2, 'learning_rate': 0.5737947884623447, 'colsample_bytree': 0.4, 'colsample_bynode': 0.5, 'n_estimators': 181, 'min_child_weight': 46, 'max_depth': 9, 'subsample': 0.7, 'random_state': 0}. Best is trial 0 with value: 0.18085571851678933.
[I 2025-12-18 22:54:22,029] Trial 1 finished with value: 0.17185514638531832 and parameters: {'lambda': 4, 'alpha': 0, 'gamma': 1, 'learning_rate': 0.7422382563833648, 'colsample_bytree': 0.7, 'colsample_bynode': 0.5, 'n_estimators': 71, 'min_child_weight': 77, 'max_depth': 9, 'subsample': 0.5, 'random_state': 0}. Best is trial 1 with value: 0.17185514638531832.
[I 2025-12-18 22:54:22,226] Trial 2 finished with value: 0.18027270994490005 and parameters: {'lambda': 2, 'alpha': 1, 'gamma': 3, 'learning_rate': 0.4708912992479439, 'colsample_bytree': 0.5, 'colsample_bynode': 1.0, 'n_estimators': 124, 'min_child_weight': 83, 'max_d

Best parameters for XGBoost: {'lambda': 3, 'alpha': 0, 'gamma': 0, 'learning_rate': 0.3770526871993918, 'colsample_bytree': 1.0, 'colsample_bynode': 0.7, 'n_estimators': 192, 'min_child_weight': 89, 'max_depth': 12, 'subsample': 1.0, 'random_state': 42}


{'Mean Squared Error': 0.0277,
 'Root Mean Squared Error': 0.1665,
 'Mean Absolute Error': 0.1311,
 'R-squared Score': 0.4165,
 'Explained Variance Score': 0.4173}

In [6]:
#### Produce the interpretability plots for the overall model features 
regressor.explain(smiles_train)

100%|██████████████████████████████████████| 4579/4579 [00:11<00:00, 393.83it/s]


Explaining model


In [7]:
# You can save the trained model
regressor.save_model(explainer_path) #pass the filename

In [8]:
regressor.load_model(explainer_path)

In [9]:
# Produce interpretability plots for individual molecules
regressor.explain_mol_atoms("CC(=O)Nc1ccc(O)cc1", atomInfo=False)

ValueError: Morgan or AccFG fingerprints are required for substructure interpretability.